`helm` is equivalent to Kubernetes as `pip` is to Python. Helm is the package management system for Kubernetes; without Helm, deploying a Kubernetes cluster means copying and pasting lots of files, and running `kubectl apply -f` over and over for the right Kubernetes deployment, services, and pods. Helm does all of this with one command.

You can also:
1. Create chart versions
2. Upgrade versions
3. Debug deployments 
4. Roll back as needed 

Helm's templating engine allows one to dynamically pass in values/data to an application, meaning you can deploy the same application in many environments using different sets of values.

### install a Helm chart in your Kubernetes cluster

`helm repo add <package> <url>` # Add a repo from the Helm Hub.

`helm repo update`

`helm repo list`

#### install the chart in the cluster 

`kubectl create ns <namespace>` # Create Kubernetes namespace.

`kubectl get ns` # Ensure it was created.

`helm install <new-user-defined-name> <repository/helm-chart-name (i.e. package from above)> -n <namespace>` # Install the chart in the cluster.

`helm ls -n <namespace>` # See all installed charts in the namespace.

### inspect charts in the Kubernetes cluster

`kubectl get all -n <namespace>` # It shows a pod, service, deployment, and replicaset that determines the number of pods we're running.

`kubectl logs <pod-name> -n <namespace>`

`kubectl port-forward svc/kube-state-metrics 8080:8080 -n metrics`

### using `helm show` commands

`helm show help`

`helm show chart <repository>/<helm-chart-name>` # Similar to what is displayed in the Helm hub.

`helm show values <repository>/<helm-chart-name> > values.yaml` # Output values redirected to a file. It contains all of the values that Kubernetes turns into objects. 

`helm delete <install-name> -n <namespace-name>`

`kubectl delete ns <namespace-name>`

### deploy an application with Helm

`helm create first-chart` creates `Chart.yaml`, `charts/`, `templates/`, and `values.yaml`. These items contain everything needed to deploy an application using a Kubernetes cluster. `Chart.yaml` contains high-level information about the chart that can be passed into the helm templating engine (ex. versioning), while the `values.yaml` contains key-value pairs that allow us to configure the containers running in a pod (ex. image repo). The `charts/` directory stores sub-charts that are dependencies of the top-level charts. The `templates/` contains the `NOTES.txt` file, which is the text displayed in the terminal when someone downloads and installs this package. It contains helm templating code. All of the files in the `templates/` directory are sent through the templating engine, where the values from `Chart.yaml` and `values.yaml` are passed in. The `_helpers.tpl` file contains code that generates variables in files within the `templates/` directory. 

#### Configmaps

A configmap is a kubernetes object that stores non-sensitive data used by pods like port numbers and environment variables. 

In [ ]:
%%file configmap.yaml

apiVersion: v1
kind: ConfigMap
metadata:
  name: first-chart-configmap
data:
  port: "8080"
  allowTesting: "true"

`helm install first-chart .`

`kubectl get cm`

`kubectl describe cm first-chart-configmap`

`helm template first-chart .` # Look over the template before running it.

`helm upgrade first-chart .` # Apply it to the cluster.

#### Secrets

Kubernetes secrets store sensitive information like passwords and SSH keys. 

In [ ]:
%%file secret.yaml

apiVersion: v1
kind: Secret
metadata:
  name: first-secret
type: Opaque
data:
  username:
  password:
 
---
# To convert to base64
username: 'admin'
password: '4w572$9sns1&!'

`echo -n 'admin' | base64`

`echo -n '4w572$9sns1&!' | base64`

In [ ]:
%%file secret.yaml

apiVersion: v1
kind: Secret
metadata:
  name: first-secret
type: Opaque
data:
  username: YWRtaW4=
  password: NHc1NzIkOXNuczEmIQ==

`helm template first-chart .`

`helm upgrade first-chart .`

`kubectl get secrets`

`kubectl describe secret first-secret`

#### Rollbacks

`helm history first-chart`

`helm rollback first-chart`

`helm rollback first-chart <revision-number>`

### Render a configmap value dynamically with Helm templating

Helm can dynamically render values.

In [ ]:
%%file configmap.yaml

apiVersion: v1
kind: ConfigMap
metadata:
  name: first-chart-configmap-{{.Chart.Version}}
data:
  port: "8080"
  allowTesting: "true"

In [ ]:
%%file Chart.yaml

apiVersion: v2
name: first-chart
description: A Helm chart for Kubernetes

# A chart can be either an 'application' or a 'library' chart.
#
# Application charts are a collection of templates that can be packaged into versioned archives
# to be deployed.
#
# Library charts provide useful utilities or functions for the chart developer. They're included as
# a dependency of application charts to inject those utilities and functions into the rendering
# pipeline. Library charts do not define any templates and therefore cannot be deployed.
type: application

# This is the chart version. This version number should be incremented each time you make changes
# to the chart and its templates, including the app version.
# Versions are expected to follow Semantic Versioning (https://semver.org/)
version: 0.1.1

# This is the version number of the application being deployed. This version number should be
# incremented each time you make changes to the application. Versions are not expected to
# follow Semantic Versioning. They should reflect the version the application is using.
appVersion: 1.16.0

In [ ]:
%%file configmap.yaml

apiVersion: v1
kind: ConfigMap
metadata:
  name: first-chart-configmap-{{.Chart.Version}}
data:
  port: "8080"
  allowTesting: "true"

`helm upgrade first-chart .`

### Using values.yaml

In [ ]:
%%file staging-values.yaml

staging:
  sample-key: sample-12345

In [ ]:
%%file values.yaml

# Default values for first-chart.
# This is a YAML-formatted file.
# Declare variables to be passed into your templates.

replicaCount: 1

image:
  repository: nginx
  pullPolicy: IfNotPresent
  # Overrides the image tag whose default is the chart appVersion.
  tag: ""

imagePullSecrets: []
nameOverride: ""
fullnameOverride: ""

serviceAccount:
  # Specifies whether a service account should be created
  create: true
  # Annotations to add to the service account
  annotations: {}
  # The name of the service account to use.
  # If not set and create is true, a name is generated using the fullname template
  name: ""

podAnnotations: {}

podSecurityContext: {}
  # fsGroup: 2000

securityContext: {}
  # capabilities:
  #   drop:
  #   - ALL
  # readOnlyRootFilesystem: true
  # runAsNonRoot: true
  # runAsUser: 1000

service:
  type: ClusterIP
  port: 80

ingress:
  enabled: false
  annotations: {}
    # kubernetes.io/ingress.class: nginx
    # kubernetes.io/tls-acme: "true"
  hosts:
    - host: chart-example.local
      paths: []
  tls: []
  #  - secretName: chart-example-tls
  #    hosts:
  #      - chart-example.local

resources: {}
  # We usually recommend not to specify default resources and to leave this as a conscious
  # choice for the user. This also increases chances charts run on environments with little
  # resources, such as Minikube. If you do want to specify resources, uncomment the following
  # lines, adjust them as necessary, and remove the curly braces after 'resources:'.
  # limits:
  #   cpu: 100m
  #   memory: 128Mi
  # requests:
  #   cpu: 100m
  #   memory: 128Mi

autoscaling:
  enabled: false
  minReplicas: 1
  maxReplicas: 100
  targetCPUUtilizationPercentage: 80
  # targetMemoryUtilizationPercentage: 80

nodeSelector: {}

tolerations: []

affinity: {}

In [ ]:
%%file values.yaml

# Default values for first-chart.
# This is a YAML-formatted file.
# Declare variables to be passed into your templates.

staging:
  sample-key: sample-12345 # Use {{ .Values.staging.sample-key }} to access. 

replicaCount: 1

image:
  repository: nginx
  pullPolicy: IfNotPresent
  # Overrides the image tag whose default is the chart appVersion.
  tag: ""

imagePullSecrets: []
nameOverride: ""
fullnameOverride: ""

serviceAccount:
  # Specifies whether a service account should be created
  create: true
  # Annotations to add to the service account
  annotations: {}
  # The name of the service account to use.
  # If not set and create is true, a name is generated using the fullname template
  name: ""

podAnnotations: {}

podSecurityContext: {}
  # fsGroup: 2000

securityContext: {}
  # capabilities:
  #   drop:
  #   - ALL
  # readOnlyRootFilesystem: true
  # runAsNonRoot: true
  # runAsUser: 1000

service:
  type: ClusterIP
  port: 80

ingress:
  enabled: false
  annotations: {}
    # kubernetes.io/ingress.class: nginx
    # kubernetes.io/tls-acme: "true"
  hosts:
    - host: chart-example.local
      paths: []
  tls: []
  #  - secretName: chart-example-tls
  #    hosts:
  #      - chart-example.local

resources: {}
  # We usually recommend not to specify default resources and to leave this as a conscious
  # choice for the user. This also increases chances charts run on environments with little
  # resources, such as Minikube. If you do want to specify resources, uncomment the following
  # lines, adjust them as necessary, and remove the curly braces after 'resources:'.
  # limits:
  #   cpu: 100m
  #   memory: 128Mi
  # requests:
  #   cpu: 100m
  #   memory: 128Mi

autoscaling:
  enabled: false
  minReplicas: 1
  maxReplicas: 100
  targetCPUUtilizationPercentage: 80
  # targetMemoryUtilizationPercentage: 80

nodeSelector: {}

tolerations: []

affinity: {}

### Helm conditionals

In [ ]:
%%file configmap.yaml

apiVersion: v1
kind: ConfigMap
metadata:
  name: first-chart-configmap-{{.Chart.Version}}
data:
  port: "8080"
  {{ if eq .Values.env "staging"}}
  allowTesting: "true"
  {{else}}
  allowTesting: "false"
  {{end}}